In [54]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from openai import OpenAI

In [ ]:
load_dotenv(override=True)
client = os.getenv('OPENAI_API_KEY')

if client:
    print(f"OpenAI API Key exists and begins {client[:8]}")
else:
    print("OpenAI API Key not set")

In [78]:
def get_security_news():
    url = "https://news.google.com/rss/search?q=security+error+threat+phishing"
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "xml")
    items = soup.find_all("item")
    return [(i.title.text, i.link.text) for i in items[:5]]

In [75]:
openai = OpenAI()

In [79]:
def summarize_article(article_text):
    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You are a cybersecurity analyst."},
            {"role": "user", "content": f"Summarize this article focusing on the security incident short:\n\n{article_text}"}
        ]
    )
    return response.choices[0].message.content

In [80]:
# Call summarizer
summary = summarize_article(article_text)
print("Summary:\n", summary)


Summary:
 A misconfigured server exposed customer data at a major tech company; security researchers detected the exposure, the company confirmed it yesterday, and no financial information was leaked.


In [59]:
def generate_pitch(summary):
    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You are a cybersecurity marketing assistant."},
            {"role": "user", "content": f"Write a brief LinkedIn-style post explaining this security incident and how Descope helps prevent such problems:\n\n{summary}"}
        ]
    )
    return response.choices[0].message.content

In [36]:
# Call pitch generator
pitch = generate_pitch(summary)
print(pitch)


Security incident reminder: A misconfigured server exposed customer data. The exposure was detected by security researchers and confirmed by the company yesterday. No financial information was leaked, but this shows how quickly sensitive data can be exposed when cloud/configuration controls aren’t tight enough.

What happened and why it matters
- A server was misconfigured, creating a pathway to customer data.
- Detection came from independent researchers and the company validated it yesterday.
- Impact: customer data was exposed; financial information remained safe. This highlights the ongoing risk from misconfigurations and the importance of robust access controls and continuous monitoring.

How Descope helps prevent these problems
- Identity-first, zero-trust access to data and apps
  - Access decisions are driven by who the user is, their device, location, and risk context, not just IPs or static permissions.
- Adaptive, risk-based authentication
  - Step-up MFA and additional veri

In [82]:
def email_pitch(summary):
    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You are a cybersecurity marketing assistant reaching out via email."},
            {"role": "user", "content": f"Write a brief marketing email to this company that we noticed this security error \
            how Descope helps prevent such problems \
            ask to reach out for a personal session:\n\n{summary}"}
        ]
    )
    return response.choices[0].message.content

In [83]:
email_p = email_pitch(summary)
print(email_p)

Subject: Prevent misconfigurations from exposing data — quick personal session?

Hello,

We recently observed a high-profile incident where a misconfigured server exposed customer data at a major tech company. While no financial information was leaked, it underscores how quickly exposure can occur. At Descope, we help teams prevent these problems with an identity-first security approach.

How Descope helps prevent this kind of risk:
- Enforce least privilege with just-in-time access and automated access revocation
- Remove passwords and reduce phishing with passwordless authentication
- Contextual, adaptive access controls that protect sensitive data
- Continuous monitoring of access patterns and automated remediation for misconfigurations
- Seamless integration with your cloud, apps, and CI/CD stacks

Would you be open to a brief personal session to review your current setup and discuss a tailored plan? I’d be happy to map practical steps to reduce exposure and align with your securit

In [61]:
def linkedin_pitch(summary):
    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You are a cybersecurity marketing assistant reaching out via email."},
            {"role": "user", "content": f"Write a concise linkedin reachout message to this company that we noticed this security error \
            how Descope helps prevent such problems \
            ask to reach out for a personal session:\n\n{summary}"}
        ]
    )
    return response.choices[0].message.content

In [51]:
linkedin = linkedin_pitch(summary)
print(linkedin)

Hi [Name], I noticed your team recently confirmed a misconfigured server exposing customer data (no financial data leaked). Descope helps prevent these issues with passwordless authentication, phishing-resistant MFA, and zero-trust access controls that enforce least privilege. If you’re open to it, I’d love to offer a quick personal session to tailor a security approach for your environment. Do you have 15 minutes this week?


In [71]:
import gradio as gr
def scrape_and_generate(api_key, url, message_type):
    # Store API key in environment for use in your logic
    os.environ["OPENAI_API_KEY"] = api_key  

    # # TODO: Replace with your scraping function
    # scraped_data = f"Scraped info from {url}"  

    # TODO: Replace with your message generation function
    if message_type == "LinkedIn message":
        message = linkedin
    else:
        message = email_p

    return message

In [ ]:
# --- Gradio UI ---

with gr.Blocks() as demo:
    gr.Markdown("# 🌐 Outreach Message Generator")
    gr.Markdown("Enter your API key, paste a website URL, and choose message type.")

    api_key = gr.Textbox(label="Enter your API Key", type="password")
    message_type = gr.Dropdown(["LinkedIn message", "Email"], label="Message Type")
    
    output = gr.Textbox(label="Generated Message", lines=10)
    submit_btn = gr.Button("Generate")

    submit_btn.click(scrape_and_generate, [api_key, url, message_type], output)

demo.launch()